In [1]:
from pathlib import Path

from agno.agent import Agent
from agno.knowledge.csv_url import CSVUrlKnowledgeBase
from agno.knowledge.csv import CSVKnowledgeBase

* Models

In [2]:
from agno.models.ollama import Ollama

ollama_model=Ollama(id="llama3.2:3b")

In [3]:
from agno.embedder.ollama import OllamaEmbedder

ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)


In [4]:
len(ollama_embedder.get_embedding("Hello world"))

768

* Vector Database

In [5]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [6]:
# Create CSV knowledge base
knowledge_base = CSVKnowledgeBase(
    path=Path("../data/employees.csv"),
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: ..\data\employees.csv

INFO     Added 2 documents to knowledge base

In [7]:
knowledge_base

CSVKnowledgeBase(reader=CSVReader(chunk=True, chunk_size=3000, separators=['\n', '\n\n', '\r', '\r\n', '\n\r', '\t', ' ', '  '], chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x0000019384722C40>), vector_db=<agno.vectordb.lancedb.lance_db.LanceDb object at 0x00000193833D4AD0>, num_documents=5, optimize_on=1000, chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x0000019384722C40>, path=WindowsPath('../data/employees.csv'))

* Agent with knowledge

In [14]:
# Create and use the agent
agent = Agent(model=ollama_model,
            knowledge=knowledge_base,  
              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
            search_knowledge=True,
            show_tool_calls=True,
            debug_mode=True,
            markdown=False)


* Tests

In [9]:
agent.run("What is the LastName of the employee called John?")

DEBUG    *********** Agent ID: c5ac9074-e8ae-4e5f-b709-182ca71ae9a0 ***********

DEBUG    *********** Session ID: c422a3b0-11ba-44d4-8513-9984431b0284 ***********

DEBUG    *********** Agent Run Start: daa9813e-0aca-425d-8562-d91410856523 ***********

DEBUG    Processing tools for model

DEBUG    Included function search_knowledge_base

...

DEBUG    ============== tool ==============

DEBUG    [                                                                                                         
           {                                                                                                       
             "name": "employees",                                                                                  
             "meta_data": {                                                                                        
               "chunk": 2,                                                                                         
               "chunk_size": 2578                                                                                  
             },                                                                                                    
             "content": " Manager, 34, 75000, 2014-12-07 167, Sarah, James, Human Resources, Compensation Analyst, 
         29, 70000, 2019-09-21 168, Anthony, Watson, Engineering, Systems Engineer, 35, 80000, 2016-06-03 169,     
         Michelle, Brooks, Marketing, SEO Manager, 31, 72000, 2020-01-16 170, Daniel, Kelly, Finance, Financial    
         Controller, 38, 90000, 2013-07-25 171, Laura, Sanders, Engineering, Network Engineer, 30, 78000,          
         2017-11-11 172, Thomas, Price, Sales, Sales Operations Manager, 33, 69000, 2015-04-19 173, Amanda,        
         Bennett, Customer Support, Customer Experience Manager, 28, 62000, 2021-08-30 174, Christopher, Wood,     
         Human Resources, HR Director, 42, 95000, 2012-10-22 175, Jessica, Barnes, Marketing, Content Editor, 27,  
         58000, 2022-02-14 176, Matthew, Ross, Engineering, Software Developer, 29, 74000, 2018-09-05 177, Susan,  
         Henderson, Finance, Auditor, 36, 83000, 2014-05-17 178, Andrew, Reyes, Marketing, Social Media            
         Coordinator, 25, 54000, 2022-06-28 179, Karen, Stevens, Customer Support, Support Technician, 31, 60000,  
         2019-03-09 180, Joshua, Morris, Engineering, Front-End Developer, 34, 80000, 2016-12-20 181, Dorothy,     
         Foster, Sales, Account Manager, 30, 66000, 2018-07-13 182, Charles, Gonzales, Human Resources, Recruitment
         Manager, 37, 85000, 2013-09-29 183, Melissa, Bryant, Marketing, Marketing Coordinator, 28, 61000,         
         2020-04-25 184, Robert, Alexander, Engineering, Back-End Developer, 32, 77000, 2017-02-08 185, Angela,    
         Russell, Finance, Financial Analyst, 35, 82000, 2015-08-16 186, Steven, Griffin, Marketing, Advertising   
         Manager, 39, 90000, 2012-11-03 187, Rebecca, Diaz, Customer Support, Client Support Specialist, 29, 63000,
         2019-05-27 188, Kevin, Hayes, Engineering, Software Engineer, 31, 76000, 2018-10-19 189, Sarah, Mendoza,  
         Human Resources, HR Assistant, 26, 58000, 2021-01-07 190, James, Fisher, Sales, Sales Representative, 28, 
         61000, 2020-09-14 191, Linda, Sullivan, Marketing, Content Creator, 34, 70000, 2016-07-22 192, Thomas,    
         Little, Engineering, DevOps Specialist, 33, 79000, 2017-04-30 193, Nancy, Bates, Finance, Senior          
         Accountant, 40, 88000, 2013-06-11 194, Daniel, Harrison, Marketing, Product Manager, 30, 75000, 2018-03-05
         195, Barbara, Stone, Customer Support, Support Lead, 32, 72000, 2019-12-18 196, Matthew, Graham,          
         Engineering, Software Architect, 38, 105000, 2014-08-09 197, Karen, West, Human Resources, HR Manager, 36,
         82000, 2015-10-27 198, Robert, Jordan, Sales, Regional Manager, 39, 89000, 2013-05-16 199, Jessica, Owens,
         Marketing, Digital Strategist, 29, 68000, 2020-02-23 200, Justin, Peterson, Engineering, Data Analyst, 31,
         76000, 2017-09-07 "                                                                                       
           },                                                                                                      
           {                                            

DEBUG    **************** METRICS ****************

DEBUG    * Time:                        0.4224s

DEBUG    **************** METRICS ******************

In [13]:
# agent.run("How old is John Doe?")
# # #Latencia 26.7 s

In [11]:
# agentic_agent = Agent(model=ollama_model,
#             knowledge=knowledge_base,  
#               # Add a tool to search the knowledge base which enables agentic RAG.
#                # This is enabled by default when `knowledge` is provided to the Agent.
#             search_knowledge=True,
#             show_tool_calls=True,
#             markdown=False)

In [12]:
# agentic_agent.run("How old is John Doe?")